# Breast Cancer Diagnosis using RBF, PNN, and GRNN (Paper Formulas)
This notebook implements **Radial Basis Function Network (RBF)**, **Probabilistic Neural Network (PNN)**, and **General Regression Neural Network (GRNN)** exactly according to the formulas from the 2004 paper, using the provided breast cancer dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.spatial.distance import cdist

# Load and preprocess dataset
df = pd.read_csv("data.csv")
df.drop(columns=["id", "Unnamed: 32"], inplace=True)
df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})

X = df.drop(columns=["diagnosis"]).values
y = df["diagnosis"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# --- RBF Network ---
class RBFNetwork:
    def __init__(self, spread=4.4):
        self.spread = spread

    def _kernel(self, X, centers):
        return np.exp(-cdist(X, centers, "sqeuclidean") / (2 * self.spread**2))

    def fit(self, X, y):
        self.centers = X
        G = self._kernel(X, self.centers)
        self.weights = np.linalg.pinv(G) @ y

    def predict(self, X):
        G = self._kernel(X, self.centers)
        y_pred = np.dot(G, self.weights)
        return np.round(y_pred).astype(int)

# --- PNN ---
class PNN:
    def __init__(self, sigma=1.0):
        self.sigma = sigma

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.classes = np.unique(y)

    def _pdf(self, X, X_class):
        m = X.shape[1]
        n = X_class.shape[0]
        const = 1 / (n * ((2 * np.pi) ** (m / 2)) * (self.sigma**m))
        dists = cdist(X, X_class, "sqeuclidean")
        return const * np.sum(np.exp(-dists / (2 * self.sigma**2)), axis=1)

    def predict(self, X):
        probs = []
        for cls in self.classes:
            probs.append(self._pdf(X, self.X_train[self.y_train == cls]))
        probs = np.array(probs)
        return self.classes[np.argmax(probs, axis=0)]

# --- GRNN ---
class GRNN:
    def __init__(self, h=3.0):
        self.h = h

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        dists = cdist(X, self.X_train, "sqeuclidean")
        weights = np.exp(-dists / (2 * self.h**2))
        numerator = weights @ self.y_train
        denominator = np.sum(weights, axis=1)
        y_pred = numerator / denominator
        return np.round(y_pred).astype(int)

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test, name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{name} Accuracy: {acc*100:.2f}%")
    print(f"Classification Report for {name}:")
    print(classification_report(y_test, y_pred))
    
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Benign", "Malignant"], yticklabels=["Benign", "Malignant"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"{name} - Confusion Matrix")
    plt.show()
    return acc

# RBF
rbf_acc = evaluate_model(RBFNetwork(spread=4.4), X_train_scaled, y_train, X_test_scaled, y_test, "RBF Network")

# PNN
pnn_acc = evaluate_model(PNN(sigma=1.0), X_train_scaled, y_train, X_test_scaled, y_test, "Probabilistic Neural Network")

# GRNN
grnn_acc = evaluate_model(GRNN(h=3.0), X_train_scaled, y_train, X_test_scaled, y_test, "General Regression Neural Network")

print("\nSummary of Accuracies:")
print(f"RBF: {rbf_acc*100:.2f}%")
print(f"PNN: {pnn_acc*100:.2f}%")
print(f"GRNN: {grnn_acc*100:.2f}%")